# Question 1

## Utilities 

In [1]:
#Imports Libraries 

# complex networks package - guide https://networkx.org/ 
import networkx as nx

# visualization packages - https://matplotlib.org/ 
import matplotlib.pyplot as plt
%matplotlib inline
from networkx.drawing.nx_agraph import graphviz_layout

# n-dinsional array tool - https://numpy.org/ 
import numpy as np

import pygraphviz

import matplotlib.colors as mcolors

import scipy

# for Notebook
%matplotlib inline


In [2]:
def draw(G, pos, measures, measure_name):
    
    nodes = nx.draw_networkx_nodes(G, pos, node_size=250, cmap=plt.cm.plasma, 
                                   node_color=list(measures.values()),
                                   nodelist=measures.keys())
    nodes.set_norm(mcolors.SymLogNorm(linthresh=0.01, linscale=1, base=10))
    # labels = nx.draw_networkx_labels(G, pos)
    edges = nx.draw_networkx_edges(G, pos)

    plt.title(measure_name)
    plt.colorbar(nodes)
    
    plt.axis('off')
    plt.show()

## Ingesting the edgelist

In [4]:
#Ingest edgelist as DiGraph

DiG = nx.read_edgelist(path = "/Users/wdaugherty/ECE 5260/ECE-5260-Graph-Based-Data-Science-for-Networked-Systems/HW6/wikispeedia.edgelist")


## Identifying max hub and authority value.


In [44]:
#Finds HITS Values

h,a = nx.hits(DiG)

print(max(h.items(), key = lambda x: x[1]))

('United_States', 0.0044443178061182665)


In [49]:
print(max(a.items(), key = lambda x: x[1]))

('United_States', 0.004444317806118269)


## Pagerank

In [7]:
#Creates a dict using the NetworkX function 
dict1 = nx.pagerank(DiG)

In [13]:
"""
Custom PageRank


Args: 
    G: Input graph
    a: alpha parameter

Outputs:
    A dictionary with the  PageRank

"""
def PageRank(G,a):

    res = {}
    return res

In [10]:
#Creates a dict using the custom PageRank function
dict2 = PageRank(DiG)

NameError: name 'PageRank' is not defined

In [9]:
#Plots the values of the two PageRank output

plt.plot(list(dict1.keys()), list(dict1.values()), list(dict2.keys()), list(dict2.values()))

NameError: name 'dict2' is not defined

# Question 2

## Generating a connected random graph


In [8]:
G = nx.random_geometric_graph(100, 0.225, 96845)


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
